In [180]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [181]:
def get_key(d, value):
    for k, v in d.items():
        if value.lower() in list(map(lambda x: x.lower(),v)):
            return k
    return value

Загружаем файлы с экспертными рейтингами и финансовым скором:

In [182]:
df_pd=pd.read_excel('CTF_External_Ratings.xlsx')
df_ordering=pd.read_excel('CTF_expert_ordering_v0.2.xlsx')

Посмотрим, для каких компаний есть данные в этих файлах:

In [183]:
print(df_pd.company_name.unique(), len(df_pd.company_name.unique()),sep='\n')

['Alpicom' 'MSB BVBA' 'SOCAR Trading SA' 'ООО СОКАР РУС'
 'Contango Trading SA' 'Glencore Energy UK Ltd' 'VTBCT'
 'RUSAL Marketing GmbH' 'Total Oil Trading']
9


In [184]:
df_ordering.company_name.unique()

array(['Gunvor Group', 'MMK Trading', 'New Stream Trading AG',
       'Gunvor S.A.', 'Alpicom', 'East Metals AG', 'MSB BVBA',
       'SOCAR Trading SA', 'INTERGRAIN SA', 'RTI LTD',
       'Contango Trading SA', 'Cetracore Energy GmbH',
       'Glencore Energy UK Ltd', 'Glencore Plc',
       'Edima Trading & Business Services AG', 'VTBCT', 'VHBV',
       'Vitol SA', 'BB Energy Trading Ltd', 'BB Energy Holding NV',
       'Newcoal Trading AG', 'AF Energy SA', 'Beringfield AG',
       'RUSAL Marketing GmbH', 'DK Trade AG', 'Total Oil Trading',
       'Westford Trade Services Ltd HK', 'MBR Metals ', 'Nizh Yug Group',
       'Lago Resources SA', 'Integral Petroleum',
       'Far East Commodities Pte Ltd', 'Ronly Ltd', 'Lafonte Commerce SA',
       'KSL AG', 'Limeniko Trade and Invest Ltd., BVI',
       'Coeclerici Commodities SA', 'Eesti Viljasalv OU', 'Enet Energy',
       'ООО ИнтерметГрупп', 'АО СПК', 'ООО РН-Бункер',
       'АО Нефтебаза Красный Яр', 'АО Чукотснаб', 'АО Ямалгосснаб',
  

названия некоторых трейдинговых компаний поменяем на имена материнских/аффилированных компаний (так как агенства присваивают рейтинги именно им):

In [185]:
d={'Gunvor':['Gunvor Group','Gunvor S.A.'],
  'EVRAZ':['East Metals AG'],
  'SOCAR':['SOCAR Trading SA','ООО Сокар Рус'],
  'RUSAL':['RTI LTD','RUSAL Marketing GmbH'],
  'Glencore':['Glencore Plc','Glencore Energy UK Ltd'],
  'BB':['BB Energy Holding NV','BB Energy Trading Ltd'],
   'Тулачермет':['Lafonte Commerce SA','Alpicom'],
  'Ros Agro':['Limeniko Trade and Invest Ltd., BVI']}

In [186]:
df_pd.company_name=np.array(list((map(lambda x: get_key(d,x),df_pd.company_name))))
df_ordering.company_name=np.array(list((map(lambda x: get_key(d,x),df_ordering.company_name))))    

In [187]:
df_pd['year']=list(map(lambda x: x.year,df_pd.report_date))
df_ordering['year']=list(map(lambda x: x.year,df_ordering.report_date))

In [188]:
#удалим дубликаты из датафрейма с рейтингами:
df_pd=df_pd.drop('report_date',axis=1).drop_duplicates()

соединим данные из двух файлов в один датафрейм:

In [189]:
result = pd.merge(df_pd, df_ordering, on=["company_name", "year"])
result = result.drop(['report_date','observation_date'],axis=1)
result

,company_name,Fitch rating,Moody's rating,S&P rating,year,экспертный рейтинг (бизнес),экспертный рейтинг (андеррайтинг),financial_score
0,Тулачермет,NaN,B2,NaN,2012,4.0,2.0,67
1,Тулачермет,NaN,B2,B-,2013,4.0,2.0,189
2,Тулачермет,NaN,B2,B-,2014,4.0,3.0,66
3,Тулачермет,B,B2,B-,2015,4.0,2.0,106
4,Тулачермет,B,B2,B-,2016,4.0,2.0,142
5,Тулачермет,B,B3,B-,2017,4.0,2.0,110
6,MSB BVBA,NaN,B3,NaN,2013,1.0,2.0,125
7,MSB BVBA,NaN,Caa1,NaN,2014,1.0,2.0,110
8,SOCAR,NaN,Ba1,BB+,2012,4.0,3.0,274
9,SOCAR,BBB-,Ba1,BB+,2013,4.0,3.0,290


Теперь нужно перевести внешние рейтинги, присвоенные агенствами, в вероятности дефолта в соответствии с построенными ранее монотонными кривыми:

In [190]:
df_curves=pd.read_excel('default_curves.xlsx')
df_curves

,Unnamed: 0,Fitch_grade,Fitch_modified_DR,Fitch_observed_DR,Moodys_grade,Moodys_modified_DR,Moodys_observed_DR,SnP_grade,SnP_modified_DR,SnP_observed_DR
0,0,AAA,0.002555,0.0012,Aaa,0.000245,0.0000,AAA,0.001543,0.0000
1,1,AA+,0.003233,0.0000,Aa1,0.000351,0.0000,AA+,0.002048,0.0000
2,2,AA,0.004089,0.0000,Aa2,0.000502,0.0000,AA,0.002720,0.0002
3,3,AA-,0.005172,0.0008,Aa3,0.000718,0.0004,AA-,0.003611,0.0003
4,4,A+,0.006543,0.0000,A1,0.001028,0.0006,A+,0.004795,0.0005
5,5,A,0.008277,0.0007,A2,0.001472,0.0004,A,0.006366,0.0005
6,6,A-,0.010470,0.0006,A3,0.002106,0.0006,A-,0.008453,0.0006
7,7,BBB+,0.013244,0.0009,Baa1,0.003014,0.0012,BBB+,0.011223,0.0010
8,8,BBB,0.016753,0.0006,Baa2,0.004314,0.0016,BBB,0.014902,0.0016
9,9,BBB-,0.021192,0.0026,Baa3,0.006174,0.0022,BBB-,0.019786,0.0025


In [191]:
def prob(rating,agency):
    if rating is np.nan:
        return np.nan
    return df_curves[df_curves[agency+'_grade']==rating][agency+'_modified_DR'].reset_index(drop=True)[0]
result['PD_Fitch'] = list(map(lambda x: prob(x,'Fitch'),result['Fitch rating']))
result["PD_Moody's"] = list(map(lambda x: prob(x,'Moodys'),result['Moody\'s rating']))
result['PD_S&P'] = list(map(lambda x: prob(x,'SnP'),result['S&P rating']))

In [192]:
result['PD_Fitch'] = list(map(lambda x: prob(x,'Fitch'),result['Fitch rating']))
result["PD_Moody's"] = list(map(lambda x: prob(x,'Moodys'),result['Moody\'s rating']))
result['PD_S&P'] = list(map(lambda x: prob(x,'SnP'),result['S&P rating']))
result['PD_mean'] = \
np.vectorize(lambda x,y,z: np.nanmean(np.array([x,y,z])))(result['PD_Fitch'],result["PD_Moody's"],result['PD_S&P'])

In [193]:
result

,company_name,Fitch rating,Moody's rating,S&P rating,year,экспертный рейтинг (бизнес),экспертный рейтинг (андеррайтинг),financial_score,PD_Fitch,PD_Moody's,PD_S&P,PD_mean
0,Тулачермет,NaN,B2,NaN,2012,4.0,2.0,67,NaN,0.037073,NaN,0.037073
1,Тулачермет,NaN,B2,B-,2013,4.0,2.0,189,NaN,0.037073,0.108415,0.072744
2,Тулачермет,NaN,B2,B-,2014,4.0,3.0,66,NaN,0.037073,0.108415,0.072744
3,Тулачермет,B,B2,B-,2015,4.0,2.0,106,0.068637,0.037073,0.108415,0.071375
4,Тулачермет,B,B2,B-,2016,4.0,2.0,142,0.068637,0.037073,0.108415,0.071375
5,Тулачермет,B,B3,B-,2017,4.0,2.0,110,0.068637,0.053058,0.108415,0.076704
6,MSB BVBA,NaN,B3,NaN,2013,1.0,2.0,125,NaN,0.053058,NaN,0.053058
7,MSB BVBA,NaN,Caa1,NaN,2014,1.0,2.0,110,NaN,0.075935,NaN,0.075935
8,SOCAR,NaN,Ba1,BB+,2012,4.0,3.0,274,NaN,0.008836,0.026271,0.017554
9,SOCAR,BBB-,Ba1,BB+,2013,4.0,3.0,290,0.021192,0.008836,0.026271,0.018766


Найдём коэффициенты линейной регрессии $\alpha * Score_{fin} + \beta = \ln({\frac{PD^{внеш}}{1-PD^{внеш}})}$ :

In [194]:
X,y =pd.DataFrame([result['financial_score'],  pd.Series(np.ones(len(result)),name='ind term')]).T, result['PD_mean']
y=np.log(y/(1-y))
reg = LinearRegression(fit_intercept=False).fit(X, y)

In [195]:
reg.coef_

array([-2.42472323e-03, -2.98458900e+00])

Теперь для каждой компании на каждый год, для которого предоставлен финансовой скор по этой компании, можем найти вероятность дефолта, используя построенную модель регрессии $PD^{TTC}=\frac{1}{1+\exp(-(\alpha * Score_{fin} + \beta))}$:


In [196]:
X=pd.DataFrame(df_ordering['financial_score'])
X['ind term']=np.ones(len(X))
df_ordering['PD_TTC']=1/(1+np.exp(-reg.predict(X)))

In [197]:
df_ordering

,company_name,report_date,observation_date,экспертный рейтинг (бизнес),экспертный рейтинг (андеррайтинг),financial_score,year,PD_TTC
0,Gunvor,2010-01-01,2010-04-01,4.0,4.0,419,2010,0.017977
1,Gunvor,2011-01-01,2011-04-01,4.0,4.0,424,2011,0.017764
2,Gunvor,2012-01-01,2012-04-01,4.0,4.0,427,2012,0.017637
3,Gunvor,2013-01-01,2013-04-01,4.0,4.0,422,2013,0.017849
4,Gunvor,2014-01-01,2014-04-01,4.0,4.0,423,2014,0.017806
...,...,...,...,...,...,...,...,...
178,ООО Королевский трубный завод,2015-01-01,2015-04-01,4.0,3.0,135,2015,0.035164
179,ООО Королевский трубный завод,2016-01-01,2016-04-01,4.0,4.0,296,2016,0.024073
180,ООО Королевский трубный завод,2017-01-01,2017-04-01,4.0,4.0,251,2017,0.026774
181,ООО Королевский трубный завод,2018-01-01,2018-04-01,4.0,4.0,322,2018,0.022635


In [198]:
X=pd.DataFrame(result['financial_score'])
X['ind term']=np.ones(len(X))
result['PD_TTC']=1/(1+np.exp(-reg.predict(X)))

In [199]:
result

,company_name,Fitch rating,Moody's rating,S&P rating,year,экспертный рейтинг (бизнес),экспертный рейтинг (андеррайтинг),financial_score,PD_Fitch,PD_Moody's,PD_S&P,PD_mean,PD_TTC
0,Тулачермет,NaN,B2,NaN,2012,4.0,2.0,67,NaN,0.037073,NaN,0.037073,0.041208
1,Тулачермет,NaN,B2,B-,2013,4.0,2.0,189,NaN,0.037073,0.108415,0.072744,0.030982
2,Тулачермет,NaN,B2,B-,2014,4.0,3.0,66,NaN,0.037073,0.108415,0.072744,0.041304
3,Тулачермет,B,B2,B-,2015,4.0,2.0,106,0.068637,0.037073,0.108415,0.071375,0.037630
4,Тулачермет,B,B2,B-,2016,4.0,2.0,142,0.068637,0.037073,0.108415,0.071375,0.034593
5,Тулачермет,B,B3,B-,2017,4.0,2.0,110,0.068637,0.053058,0.108415,0.076704,0.037280
6,MSB BVBA,NaN,B3,NaN,2013,1.0,2.0,125,NaN,0.053058,NaN,0.053058,0.035996
7,MSB BVBA,NaN,Caa1,NaN,2014,1.0,2.0,110,NaN,0.075935,NaN,0.075935,0.037280
8,SOCAR,NaN,Ba1,BB+,2012,4.0,3.0,274,NaN,0.008836,0.026271,0.017554,0.025358
9,SOCAR,BBB-,Ba1,BB+,2013,4.0,3.0,290,0.021192,0.008836,0.026271,0.018766,0.024417


Осталось с помощью мастер-шкалы перевести найденные вероятности дефолта во внутренние рейтинги.
Загрузим файл со шкалой:

In [200]:
df_scale=pd.read_excel('scale.xlsx')
df_scale

,Rating,upper bound (%),lower bound (%)
0,AAA,0.01,0.00
1,AA+,0.02,0.01
2,AA,0.03,0.02
3,AA-,0.05,0.03
4,A+,0.08,0.05
5,A,0.12,0.08
6,A-,0.19,0.12
7,BBB+,0.30,0.19
8,BBB,0.46,0.30
9,BBB-,0.73,0.46


Добавим в датафреймы столбцы, соответствующие внутренним рейтингам, получаемым из $PD^{TTC}$ с помощью мастер-шкалы:

In [201]:
def scale_rating(pd):
    pd=pd*100
    if pd>=99.0: return 'Default'
    else: return df_scale.Rating[df_scale[df_scale['upper bound (%)']>pd].index[0]]

In [202]:
df_ordering['internal rating'] = np.vectorize(scale_rating)(df_ordering['PD_TTC'])
result['internal rating'] = np.vectorize(scale_rating)(result['PD_TTC'])

In [203]:
df_ordering

,company_name,report_date,observation_date,экспертный рейтинг (бизнес),экспертный рейтинг (андеррайтинг),financial_score,year,PD_TTC,internal rating
0,Gunvor,2010-01-01,2010-04-01,4.0,4.0,419,2010,0.017977,BB-
1,Gunvor,2011-01-01,2011-04-01,4.0,4.0,424,2011,0.017764,BB
2,Gunvor,2012-01-01,2012-04-01,4.0,4.0,427,2012,0.017637,BB
3,Gunvor,2013-01-01,2013-04-01,4.0,4.0,422,2013,0.017849,BB-
4,Gunvor,2014-01-01,2014-04-01,4.0,4.0,423,2014,0.017806,BB-
...,...,...,...,...,...,...,...,...,...
178,ООО Королевский трубный завод,2015-01-01,2015-04-01,4.0,3.0,135,2015,0.035164,B+
179,ООО Королевский трубный завод,2016-01-01,2016-04-01,4.0,4.0,296,2016,0.024073,BB-
180,ООО Королевский трубный завод,2017-01-01,2017-04-01,4.0,4.0,251,2017,0.026774,BB-
181,ООО Королевский трубный завод,2018-01-01,2018-04-01,4.0,4.0,322,2018,0.022635,BB-


In [204]:
result

,company_name,Fitch rating,Moody's rating,S&P rating,year,экспертный рейтинг (бизнес),экспертный рейтинг (андеррайтинг),financial_score,PD_Fitch,PD_Moody's,PD_S&P,PD_mean,PD_TTC,internal rating
0,Тулачермет,NaN,B2,NaN,2012,4.0,2.0,67,NaN,0.037073,NaN,0.037073,0.041208,B+
1,Тулачермет,NaN,B2,B-,2013,4.0,2.0,189,NaN,0.037073,0.108415,0.072744,0.030982,B+
2,Тулачермет,NaN,B2,B-,2014,4.0,3.0,66,NaN,0.037073,0.108415,0.072744,0.041304,B+
3,Тулачермет,B,B2,B-,2015,4.0,2.0,106,0.068637,0.037073,0.108415,0.071375,0.037630,B+
4,Тулачермет,B,B2,B-,2016,4.0,2.0,142,0.068637,0.037073,0.108415,0.071375,0.034593,B+
5,Тулачермет,B,B3,B-,2017,4.0,2.0,110,0.068637,0.053058,0.108415,0.076704,0.037280,B+
6,MSB BVBA,NaN,B3,NaN,2013,1.0,2.0,125,NaN,0.053058,NaN,0.053058,0.035996,B+
7,MSB BVBA,NaN,Caa1,NaN,2014,1.0,2.0,110,NaN,0.075935,NaN,0.075935,0.037280,B+
8,SOCAR,NaN,Ba1,BB+,2012,4.0,3.0,274,NaN,0.008836,0.026271,0.017554,0.025358,BB-
9,SOCAR,BBB-,Ba1,BB+,2013,4.0,3.0,290,0.021192,0.008836,0.026271,0.018766,0.024417,BB-


В качестве признаков для модели регрессии можно также использовать предоставленные нам экспертные ранжирования компаний:

In [212]:
X,y = result[['financial_score','экспертный рейтинг (андеррайтинг)','экспертный рейтинг (бизнес)']].copy(), result['PD_mean']
X['ind term'] = np.ones(len(X))
y=np.log(y/(1-y))
reg = LinearRegression(fit_intercept=False).fit(X, y)

X = result[['financial_score','экспертный рейтинг (андеррайтинг)','экспертный рейтинг (бизнес)']].copy()
X['ind term'] = np.ones(len(X))
result['PD_TTC_2'] = 1/(1+np.exp(-reg.predict(X)))

df_ordering_nonan=df_ordering.dropna().copy()
X = df_ordering_nonan[['financial_score','экспертный рейтинг (андеррайтинг)','экспертный рейтинг (бизнес)']].copy()
X['ind term'] = np.ones(len(X))
df_ordering_nonan['PD_TTC_2'] = 1/(1+np.exp(-reg.predict(X)))

In [213]:
df_ordering_nonan['internal rating 2'] = np.vectorize(scale_rating)(df_ordering_nonan['PD_TTC_2'])
result['internal rating 2'] = np.vectorize(scale_rating)(result['PD_TTC_2'])

In [214]:
df_ordering_nonan

,company_name,report_date,observation_date,экспертный рейтинг (бизнес),экспертный рейтинг (андеррайтинг),financial_score,year,PD_TTC,internal rating,PD_TTC_2,internal rating 2
0,Gunvor,2010-01-01,2010-04-01,4.0,4.0,419,2010,0.017977,BB-,0.014165,BB
1,Gunvor,2011-01-01,2011-04-01,4.0,4.0,424,2011,0.017764,BB,0.014103,BB
2,Gunvor,2012-01-01,2012-04-01,4.0,4.0,427,2012,0.017637,BB,0.014067,BB
3,Gunvor,2013-01-01,2013-04-01,4.0,4.0,422,2013,0.017849,BB-,0.014128,BB
4,Gunvor,2014-01-01,2014-04-01,4.0,4.0,423,2014,0.017806,BB-,0.014116,BB
...,...,...,...,...,...,...,...,...,...,...,...
178,ООО Королевский трубный завод,2015-01-01,2015-04-01,4.0,3.0,135,2015,0.035164,B+,0.028838,B+
179,ООО Королевский трубный завод,2016-01-01,2016-04-01,4.0,4.0,296,2016,0.024073,BB-,0.015758,BB
180,ООО Королевский трубный завод,2017-01-01,2017-04-01,4.0,4.0,251,2017,0.026774,BB-,0.016385,BB
181,ООО Королевский трубный завод,2018-01-01,2018-04-01,4.0,4.0,322,2018,0.022635,BB-,0.015407,BB


In [215]:
result

,company_name,Fitch rating,Moody's rating,S&P rating,year,экспертный рейтинг (бизнес),экспертный рейтинг (андеррайтинг),financial_score,PD_Fitch,PD_Moody's,PD_S&P,PD_mean,PD_TTC,internal rating,PD_TTC_2,internal rating 2,PD_TTC_3,internal rating 3
0,Тулачермет,NaN,B2,NaN,2012,4.0,2.0,67,NaN,0.037073,NaN,0.037073,0.041208,B+,0.048293,B,0.051201,B
1,Тулачермет,NaN,B2,B-,2013,4.0,2.0,189,NaN,0.037073,0.108415,0.072744,0.030982,B+,0.043591,B+,0.057143,B
2,Тулачермет,NaN,B2,B-,2014,4.0,3.0,66,NaN,0.037073,0.108415,0.072744,0.041304,B+,0.030588,B+,0.020044,BB-
3,Тулачермет,B,B2,B-,2015,4.0,2.0,106,0.068637,0.037073,0.108415,0.071375,0.037630,B+,0.046740,B,0.053034,B
4,Тулачермет,B,B2,B-,2016,4.0,2.0,142,0.068637,0.037073,0.108415,0.071375,0.034593,B+,0.045348,B,0.054781,B
5,Тулачермет,B,B3,B-,2017,4.0,2.0,110,0.068637,0.053058,0.108415,0.076704,0.037280,B+,0.046583,B,0.053225,B
6,MSB BVBA,NaN,B3,NaN,2013,1.0,2.0,125,NaN,0.053058,NaN,0.053058,0.035996,B+,0.032602,B+,0.110993,CCC
7,MSB BVBA,NaN,Caa1,NaN,2014,1.0,2.0,110,NaN,0.075935,NaN,0.075935,0.037280,B+,0.033021,B+,0.109592,CCC
8,SOCAR,NaN,Ba1,BB+,2012,4.0,3.0,274,NaN,0.008836,0.026271,0.017554,0.025358,BB-,0.025602,BB-,0.024324,BB-
9,SOCAR,BBB-,Ba1,BB+,2013,4.0,3.0,290,0.021192,0.008836,0.026271,0.018766,0.024417,BB-,0.025253,BB-,0.024688,BB-


Попробуем применить полиномиальную регрессию:

In [233]:
poly = PolynomialFeatures(2)

X,y = result[['financial_score','экспертный рейтинг (андеррайтинг)','экспертный рейтинг (бизнес)']].copy(), result['PD_mean']
poly.fit_transform(X)
y=np.log(y/(1-y))
reg = LinearRegression(fit_intercept=False).fit(X, y)

X = result[['financial_score','экспертный рейтинг (андеррайтинг)','экспертный рейтинг (бизнес)']].copy()
poly.fit_transform(X)
result['PD_TTC_3'] = 1/(1+np.exp(-reg.predict(X)))

X = df_ordering_nonan[['financial_score','экспертный рейтинг (андеррайтинг)','экспертный рейтинг (бизнес)']].copy()
poly.fit_transform(X)
df_ordering_nonan['PD_TTC_3'] = 1/(1+np.exp(-reg.predict(X)))

df_ordering_nonan['internal rating 3'] = np.vectorize(scale_rating)(df_ordering_nonan['PD_TTC_3'])
result['internal rating 3'] = np.vectorize(scale_rating)(result['PD_TTC_3'])

Распределение компаний по внутренним рейтингам:

In [232]:
df_dist=pd.DataFrame([df_ordering_nonan['internal rating 3'].value_counts(),df_ordering_nonan['internal rating 2'].value_counts(),df_ordering_nonan['internal rating'].value_counts()]).T.fillna(0)
df_dist.astype(int)

,internal rating 3,internal rating 2,internal rating
BB-,41,37,52
BB,40,58,48
B,32,22,18
BB+,22,12,6
B-,16,0,0
B+,12,40,45
CCC,4,0,0
CC,2,0,0


### Выводы:

Низкое качество модели может быть связано со следующими особенностями:

    1) выборка, по которой подбирались параметры регрессии слишком мала;
    2) финансовый скор и экспертные ретийнги имеются для трединговых компаний, а внешние рейтинги присвоены материнским компаниям (что приводит даже к наличию одинаковых внешних рейтингов при существенно отличающемся скор-балле в ситуации, когда у одной крупной компании есть несколько трейдинговых структур).
    3) используются несовершенные монотонные кривые для нахождения вероятностей дефолта по внешним рейтингам